In [1]:
%config IPCompleter.greedy=True
import 
importlib._setup()

In [2]:
# %load_ext snakeviz

In [3]:
%load_ext autoreload
%autoreload 2

In [5]:
# native modules
import sys
import datetime
from os import path, listdir
from random import choice, sample
import re
import itertools
# from importlib import reload

In [6]:
# data study
import pandas as pd
import numpy as np
from copy import deepcopy
import requests

In [ ]:
# multithreading
# import concurrent.futures
# import threading

In [7]:
# charting
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Recon project
# sys.path.append(r'C:\Users\kklekota\PycharmProjects\Recon')
# import utils.xml_parser as parser

In [ ]:
# mkt data project
# sys.path.append(r'C:\Users\kklekota\PycharmProjects\mkt_data')
# from mkt_data.mkt_data import const, helper
# import mkt_data.mkt_data as md

In [ ]:
# macs service
# from macs_service import Client, generate_json_data
# client = Client("http://leviathan33:5000")

In [ ]:
# %%HTML
# <iframe src="http://ci-macsi/jenkins/view/MACS%20services%20status/" 
#  width="600"  height="450" frameborder="0" scrolling="no"></iframe>

In [ ]:
# service project
# sys.path.append(r'C:\Users\kklekota\PycharmProjects\Service')
# import foxutils as fu
# import h5helper
# import mxgrid_replay as replay

In [25]:
def print_repl_quality(label, ref_label, ref_value, repl_label, repl_value, is_nominal_adj=False):
    mult = nominal if is_nominal_adj else 1.0
    REPL_FMT_STR = '{}:\n\t{:<15}{}\n\t{:<15}{}\n\t{:<15}{}'
    print(REPL_FMT_STR.format(label, ref_label, mult * ref_value, 
                              repl_label, mult * repl_value, 
                              "difference", mult * (ref_value - repl_value)))

In [34]:
from fox_toolbox.utils import xml_parser
from fox_toolbox.utils.rates import Swaption

In [123]:
inps = (
    r"D:\Work\Issues\Mizuho\20190313_hedge_curves\irsmform_mx13548vm_MAINHEDGE.xml",
    r"D:\Work\Issues\Mizuho\20190313_hedge_curves\irsmform_mx13548vm_no_hedge.xml",
)
outs = (
    r"D:\Work\Issues\Mizuho\20190313_hedge_curves\irsmout_mx13548vm_MAINHEDGE.xml",
    r"D:\Work\Issues\Mizuho\20190313_hedge_curves\irsmout_mx13548vm_no_hedge.xml",
)

In [131]:
id_ = 1
inp, out = inps[id_], outs[id_]

In [132]:
(_, irsmform), (_, irsmout) = tuple(map(xml_parser.get_xml, (inp, out)))

In [133]:
main_curve, sprd_curves = xml_parser.get_curves(irsmform)
fwd_curve = sprd_curves[0]

In [134]:
cal_bskt = list(xml_parser.get_calib_basket(irsmout))

In [135]:
# s1 = pd.Series([f.fwd for f in cal_bskt], index=[f.expiry for f in cal_bskt], name='HEDGE')
s2 = pd.Series([f.fwd for f in cal_bskt], index=[f.expiry for f in cal_bskt], name='MAIN')

In [58]:
for i, cal_instr in enumerate(cal_bskt):
    fwd = cal_instr.get_swap_rate(main_curve, fwd_curve=fwd_curve)
    macs_fwd = cal_instr.fwd
    print_repl_quality(f'Forward {i+1}', 'macs', macs_fwd, 'replication', fwd)

Forward 1:
	macs           0.0106814541926
	replication    0.010681454192580837
	difference     1.9163490239115788e-14
Forward 2:
	macs           0.0123241526962
	replication    0.012324152696253213
	difference     -5.321264262558856e-14
Forward 3:
	macs           0.0137941858399
	replication    0.013794185839884222
	difference     1.5779044737485037e-14
Forward 4:
	macs           0.0152081518786
	replication    0.015208151878594789
	difference     5.2111093218343285e-15
Forward 5:
	macs           0.0165271461454
	replication    0.016527146145450602
	difference     -5.060188379424346e-14
Forward 6:
	macs           0.0177622709212
	replication    0.017762270921200536
	difference     -5.377642775528102e-16
Forward 7:
	macs           0.0189102409738
	replication    0.01891024097361195
	difference     1.880509636897898e-13
Forward 8:
	macs           0.0199551820336
	replication    0.019955182033574652
	difference     2.5347779430973105e-14
Forward 9:
	macs           0.0207638654457
	replic

In [59]:
def truncate_date(date):
    return round(float(date), 6)

def get_cal_instr_dates(cal_instr: Swaption):
    dates = [cal_instr.expiry, cal_instr.start_date] + list(cal_instr.payment_dates)
    return set(map(truncate_date, dates))

from functools import reduce
cal_bskt_dates = reduce(set.union, map(get_cal_instr_dates, cal_bskt))

In [60]:
curve_pillars = set(map(truncate_date, main_curve.curve_pillars))

In [61]:
cal_bskt_dates - curve_pillars

set()

In [66]:
list(irsmform.find('.//Formula').itertext())

[' 36 ; 9.498630 ; Basic ; self.Underlying=self.Underlying+1000000.000000*(Libor(0,9.509589,10.013699, 0.511111111111,tenor=6)*0.508333333333)*Discount(0, 9.498630, 10.010959)*Pay(0, 1.000000, 9.498630, 0, 1020, ExerciseDate = 9.000000)  ']

In [67]:
formulas = [f.text.strip() for f in irsmform.iterfind('.//Formula')]

In [70]:
formula_dates = set(truncate_date(float(f.split(';')[1])) for f in formulas)

In [72]:
formula_dates - curve_pillars

set()

In [141]:
FLA_RE = re.compile(('Libor\(0,(\d+\.\d+),(\d+\.\d+),[^\)]*\)'
                     '|Discount\(0,\s*(\d+\.\d+),\s*(\d+\.\d+)\)'
                     '|Pay\(0,\s*\d+\.\d+,\s*(\d+\.\d+),[^\)]*\)'))

def get_formula_dates(fla):
    res = FLA_RE.findall(fla)
    return set(map(truncate_date, (f for f in chain(*res) if f)))
    
payoff_dates = reduce(set.union, map(get_formula_dates, formulas))

In [122]:
libor_dates - curve_pillars

set()

In [142]:
payoff_dates = set.union(payoff_dates, cal_bskt_dates)
max(payoff_dates)

10.013699

In [138]:
df = pd.DataFrame([s1, s2]).T

In [140]:
df.HEDGE - df.MAIN

0.99726    3.142000e-10
2.00000    2.251000e-10
2.99726    2.583000e-10
3.99726    3.032000e-10
5.00274    4.134000e-10
6.00274    3.538000e-10
7.00274    2.359000e-10
8.00274    4.100000e-11
9.00000    8.250000e-11
dtype: float64